Associated Stack Overflow question: [Optimizing Lat/Lon Extraction from Astropy's GeocentricTrueEcliptic SkyCoord](https://stackoverflow.com/q/78324819/7758804)

The three functions presented below calculate latitude and longitude, as radians, using `x`, `y`, and `z` from the `astropy.coordinates.sky_coordinate.SkyCoord` object.

There is a `numpy` option, and two [`numba`](https://numba.pydata.org/) enhanced versions-one using the JIT and the other adding parallel processing, which offer varying improvements.

- **`@jit`**: [Compiling Python code with `@jit`](https://numba.readthedocs.io/en/stable/user/jit.html)
- **`cache`**: [Cache the compiled function on disk](https://numba.readthedocs.io/en/stable/user/jit.html#cache)
- **`parallel`**: [Automatic parallelization](https://numba.readthedocs.io/en/stable/user/jit.html#parallel)

## Results Summary

This analysis demonstrates improved performance when using Numba-enhanced functions for calculating spherical coordinates from an Astropy `SkyCoord` object in the `GeocentricTrueEcliptic` frame compared to directly accessing `.lat` and `.lon` properties.

- **Detailed Time Measurements**:
  - Direct access to `.lat` and `.lon` properties took about 17.73 microseconds (`8.89 µs` for `.lat` and `8.84 µs` for `.lon`).
  - Retrieving `x`, `y`, and `z` values directly (`467 ns`, `450 ns`, and `458 ns` respectively) summed to approximately 1.375 microseconds.
  - The `cartesian_to_spherical_numba` computation took 267 nanoseconds.

- **Combining the time for direct value retrieval and computation**:
  - Total time for Numba-enhanced computation (`cartesian_to_spherical_numba`): `1.375 µs` (value retrieval) + `267 ns` = `1.642 µs`.
  - This is about 10.8 times faster than the direct access method for `.lat` and `.lon` (`17.73 µs / 1.642 µs ≈ 10.8`).

These results demonstrate the gains achieved by using JIT compilation. This is further enhanced by the use of parallelization, beneficial for arrays of coordinates.

## Relevant Imports

In [ ]:
from astropy.coordinates import SkyCoord, GeocentricTrueEcliptic, ICRS, Angle
import astropy.units as u
from astropy.time import Time
import numpy as np
from numba import jit, prange
from typing import Tuple, Union

- Python: 3.12.0
- Astropy Version: 6.0.1
- Numba Version: 0.59.1
- Numpy Version: 1.26.4

## Initial Code

In [ ]:
# International Celestial Reference System coordinates
coo_icrs = SkyCoord(150*u.deg, 19*u.deg, frame=ICRS(), obstime=Time(2460791., format="jd"))
# Ecliptic coordinates
ecliptic_coords = coo_icrs.transform_to(GeocentricTrueEcliptic)

print(ecliptic_coords.lat, ecliptic_coords.lon)

In [ ]:
coo_icrs

In [ ]:
ecliptic_coords

In [ ]:
type(coo_icrs.transform_to(GeocentricTrueEcliptic))

In [ ]:
%timeit ecliptic_coords.lat
%timeit ecliptic_coords.lon
%timeit ecliptic_coords._sky_coord_frame._data
%timeit ecliptic_coords._sky_coord_frame._data.xyz.value
%timeit ecliptic_coords._sky_coord_frame._data.x.value
%timeit ecliptic_coords._sky_coord_frame._data.y.value
%timeit ecliptic_coords._sky_coord_frame._data.z.value
%timeit ecliptic_coords.cartesian.xyz.value
%timeit ecliptic_coords.cartesian.x.value

`[6d21m12.19206726s] [145d28m31.99626325s]`

### `timeit` for various functions

Accessing `._sky_coord_frame._data.x.value` is significantly faster, taking only 467 ns, compared to `6.14 µs` when using `.cartesian.x.value`. However, the use of a single underscore (`_`) prefix in the names of attributes or methods signifies they are intended to be "protected" or for internal use. See [this question](https://stackoverflow.com/q/1301346/7758804), [Private Variables](https://docs.python.org/3/tutorial/classes.html#private-variables), and [Public vs Non-Public Members](https://realpython.com/python-classes/#public-vs-non-public-members) for additional details.

```python
%timeit ecliptic_coords.lat
8.89 µs ± 132 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
%timeit ecliptic_coords.lon
8.84 µs ± 413 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
%timeit ecliptic_coords._sky_coord_frame._data
76.5 ns ± 0.507 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
%timeit ecliptic_coords._sky_coord_frame._data.xyz.value
15.8 µs ± 351 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
%timeit ecliptic_coords._sky_coord_frame._data.x.value
467 ns ± 10.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
%timeit ecliptic_coords._sky_coord_frame._data.y.value
450 ns ± 3.89 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
%timeit ecliptic_coords._sky_coord_frame._data.z.value
458 ns ± 6.94 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
%timeit ecliptic_coords.cartesian.xyz.value
25.1 µs ± 409 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
%timeit ecliptic_coords.cartesian.x.value
6.14 µs ± 215 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
```

## Code to Calculate Longitude and Latitude

In [ ]:
def cartesian_to_spherical_numpy(
    x: Union[np.ndarray, np.float64],
    y: Union[np.ndarray, np.float64],
    z: Union[np.ndarray, np.float64]
    ) -> Tuple[np.ndarray, np.ndarray]:
    """
    Convert Cartesian coordinates to spherical coordinates using NumPy.

    Args:
    x (Union[np.ndarray, np.float64]): X-coordinates in Cartesian.
    y (Union[np.ndarray, np.float64]): Y-coordinates in Cartesian.
    z (Union[np.ndarray, np.float64]): Z-coordinates in Cartesian.

    Returns:
    Tuple[np.ndarray, np.ndarray]: Tuple of longitude and latitude arrays in radians.
    """
    lon = np.arctan2(y, x)  # Compute longitude using arctan2 for proper handling of quadrant.
    hypot = np.sqrt(x**2 + y**2)  # Compute the hypotenuse of x and y to use for latitude calculation.
    lat = np.arctan2(z, hypot)  # Compute latitude as the angle from the positive z-axis.
    return lon, lat


@jit(nopython=True, cache=True)
def cartesian_to_spherical_numba(
    x: Union[np.ndarray, np.float64],
    y: Union[np.ndarray, np.float64],
    z: Union[np.ndarray, np.float64]
    ) -> Tuple[np.ndarray, np.ndarray]:
    """
    Convert Cartesian coordinates to spherical coordinates using Numba for JIT compilation to speed up the computation.
    
    Args:
    x (Union[np.ndarray, np.float64]): X-coordinates in Cartesian.
    y (Union[np.ndarray, np.float64]): Y-coordinates in Cartesian.
    z (Union[np.ndarray, np.float64]): Z-coordinates in Cartesian.

    Returns:
    Tuple[np.ndarray, np.ndarray]: Tuple of longitude and latitude arrays in radians.
    """
    lon = np.arctan2(y, x)  # Computation of longitude.
    hypot = np.sqrt(x**2 + y**2)  # Computation of the hypotenuse for latitude calculation.
    lat = np.arctan2(z, hypot)  # Computation of latitude.
    return lon, lat


@jit(nopython=True, parallel=True, cache=True)
def cartesian_to_spherical_numba_parallel(x: np.ndarray, y: np.ndarray, z: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Convert Cartesian coordinates to spherical coordinates with both parallel processing and JIT compilation
    This version leverages multiple CPU cores to handle large arrays more efficiently.

    Args:
    x (np.ndarray): X-coordinates in Cartesian.
    y (np.ndarray): Y-coordinates in Cartesian.
    z (np.ndarray): Z-coordinates in Cartesian.

    Returns:
    Tuple[np.ndarray, np.ndarray]: Tuple of longitude and latitude arrays in radians.
    """
    lon = np.empty_like(x)  # Preallocate the array for longitude.
    lat = np.empty_like(x)  # Preallocate the array for latitude.
    for i in prange(x.size):  # Use parallel range (prange) to distribute loop iterations across multiple threads.
        lon[i] = np.arctan2(y[i], x[i])  # Compute longitude for each element in parallel.
        hypot = np.sqrt(x[i]**2 + y[i]**2)  # Compute the hypotenuse for each element in parallel.
        lat[i] = np.arctan2(z[i], hypot)  # Compute latitude for each element in parallel.
    return lon, lat

### Usage

#### Testing with `x`, `y`, and `z` as `np.float64`

In [ ]:
# Ensure Numba function is compiled before timing - only used for testing - not required when using functions in a loop
_ = cartesian_to_spherical_numba(np.array([1.0]), np.array([1.0]), np.array([1.0]))

# extract x, y, and z values as numpy.float64
x_ecl = ecliptic_coords._sky_coord_frame._data.x.value
y_ecl = ecliptic_coords._sky_coord_frame._data.y.value
z_ecl = ecliptic_coords._sky_coord_frame._data.z.value

# time the non-parallel functions
%timeit cartesian_to_spherical_numpy(x_ecl, y_ecl, z_ecl)
%timeit cartesian_to_spherical_numba(x_ecl, y_ecl, z_ecl)

In [ ]:
cartesian_to_spherical_numpy(x_ecl, y_ecl, z_ecl)

In [ ]:
cartesian_to_spherical_numba(x_ecl, y_ecl, z_ecl)

#### Testing with `x`, `y`, and `z` as `np.ndarray`

Use if `x`, `y`, and `z` values can be configured into three arrays of length `n`.

In [ ]:
# Generate test data
n = 1000000
x = np.random.uniform(-1, 1, n)
y = np.random.uniform(-1, 1, n)
z = np.random.uniform(-1, 1, n)

# Ensure Numba function is compiled before timing
_ = cartesian_to_spherical_numba(np.array([1.0]), np.array([1.0]), np.array([1.0]))
_ = cartesian_to_spherical_numba_parallel(np.array([1.0]), np.array([1.0]), np.array([1.0]))

# time all the functions
%timeit cartesian_to_spherical_numpy(x, y, z)
%timeit cartesian_to_spherical_numba(x, y, z)
%timeit cartesian_to_spherical_numba_parallel(x, y, z)

## Radians and Degree, Minute, Seconds

`.lat` and `.lon` return values formatted as (degrees, minutes, seconds), while the implemented functions return values as radians, which are natively used by numpy and numba. The following code presents additional format options, which are not considered in the timing analysis.

### Converting Radians to Degrees

In [ ]:
@jit(nopython=True, cache=True)
def cartesian_to_spherical_numba_deg(x, y, z):
    lon = np.arctan2(y, x)
    hypot = np.sqrt(x**2 + y**2)
    lat = np.arctan2(z, hypot)
    return np.degrees(lon), np.degrees(lat)

In [ ]:
cartesian_to_spherical_numba_deg(x_ecl, y_ecl, z_ecl)

In [ ]:
@jit(nopython=True, parallel=True)
def cartesian_to_spherical_numba_parallel_deg(x, y, z):
    lon = np.empty_like(x)
    lat = np.empty_like(x)
    for i in prange(x.size):
        lon[i] = np.arctan2(y[i], x[i])
        hypot = np.sqrt(x[i]**2 + y[i]**2)
        lat[i] = np.arctan2(z[i], hypot)
    return np.degrees(lon), np.degrees(lat)

### Formatting as Degrees, Minutes, and Seconds

In [ ]:
# Example radians-to-degree conversion
lon_rad = np.array([2.61799388])  # longitude in radians
lat_rad = np.array([0.33161256])  # latitude in radians

# Convert radians to degrees
lon_deg, lat_deg = np.degrees(lon_rad), np.degrees(lat_rad)

# Create Angle objects and format them
lon_angle = Angle(lon_deg, u.degree)
lat_angle = Angle(lat_deg, u.degree)

# Print formatted angles
print(lon_angle.to_string(unit=u.degree, sep=('d', 'm', 's')))
print(lat_angle.to_string(unit=u.degree, sep=('d', 'm', 's')))